In [31]:
import pandas as pd
import os

In [32]:
if os.path.exists("sp500.csv"):
    sp500 = pd.read_csv("sp500.csv", index_col=0)
else:
    sp500 = yf.Ticker("^GSPC")
    sp500 = sp500.history(period="max")
    sp500.to_csv("sp500.csv")

In [33]:
#sp500.plot.line(y='close', use_index=True)

del sp500["Dividends"]
del sp500["Stock Splits"]

In [34]:
#create columns: "tomorrow" (next day price) and "target" (boolean - greater than tm?)
sp500["Tomorrow"] = sp500["Close"].shift(-1)
sp500["Target"] = (sp500["Tomorrow"] > sp500["Close"]).astype(int)

In [35]:
#remove data before 1990
sp500 = sp500.loc["1990-01-01":].copy() #.copy for 

In [36]:
#machine learning model: Random Forest Classifier 
#harder to overfit, quick, can pick up non-linear tendencies 

#random_state: random # predicted will be in predicatbel sequence 
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, min_samples_split=100, 
                              random_state=1)
train = sp500.iloc[:-100]
test = sp500.iloc[-100:]

predictors = ["Close", "Volume", "Open", "High", "Low"]
model.fit(train[predictors], train["Target"])

RandomForestClassifier(min_samples_split=100, random_state=1)

In [37]:
#check accuracy
from sklearn.metrics import precision_score

preds = model.predict(test[predictors])
preds = pd.Series(preds, index=test.index)

In [38]:
#preds

In [39]:
precision_score(test["Target"], preds)
#40% - not good, shouldn't train on this 

0.42857142857142855

In [40]:
combined = pd.concat([test["Target"], preds], axis=1)
combined.plot()

In [41]:
def predict(train, test, predictors, model): 
    model.fit(train[predictors], train["Target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined


In [42]:
#tra
def backtest(data, model, predictors, start=2500, step=250): 
    all_predictors = []
    
    for i in range(start, data.shape[0], step): 
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    return pd.concat(all_predictions)

In [30]:
predictions = backtest(sp500, model, predictors)

KeyError: 'Target'